In [1]:
!pip install gpt-2-simple > log

In [2]:
import gpt_2_simple as gpt2
import os
from datetime import datetime 
import tensorflow as tf
from google.cloud import storage
os.chdir('..')

In [3]:
model_name="124M"
checkpoint_dir = 'checkpoint'
file_name = "data/songs_data.txt"
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
bucket_name = "poem-generator-checkpoints"
run_name='disco_polo_gen'

In [4]:
if not os.path.exists(os.path.join('models', model_name)):
    gpt2.download_gpt2(model_name=model_name)

In [ ]:
sess = gpt2.start_tf_sess() 
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              reuse=False,
              steps=15000,
              run_name=run_name,
              print_every=10,
              sample_every=200,
              save_every=100, 
              only_train_transformer_layers = True,
              accumulate_gradients = 1,
              overwrite=True
              )

2022-03-26 14:00:12.332838: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-26 14:00:12.333664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-03-26 14:00:12.333699: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-26 14:00:12.333728: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pytorch-1-10-20220326-144724): /proc/driver/nvidia/version does not exis

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:09<00:00,  9.96s/it]


dataset has 3432165 tokens
Training...
[10 | 95.59] loss=2.83 avg=2.83


In [ ]:
bucket = storage.Client().get_bucket(bucket_name)
for f_name in os.listdir(os.path.join(checkpoint_dir, run_name)):
    bucket.blob(os.path.join(timestamp, checkpoint_dir, run_name, f_name))\
                .upload_from_filename(os.path.join(checkpoint_dir, run_name, f_name))